In [10]:
import tweepy
import time
import os
import json
# ADD OTHER PACKAGES WE WILL NEED

**Task:** Load the values of access tokens and keys from environmental variables to python variables

In [12]:
consumer_key = os.environ['TWITTER_API']
consumer_secret = os.environ['TWITTER_API_SECRET']
access_token = os.environ['TWITTER_ACCESS']
access_token_secret = os.environ['TWITTER_ACCESS_SECRET']
bearer_token = os.environ['TWITTER_BEARER']

In [13]:
# Used the following as an example:
#   https://dev.to/suhemparack/getting-covid-19-tweets-using-the-twitter-api-v2-5b19
#   https://stackoverflow.com/questions/33498975/unable-to-stop-streaming-in-tweepy-after-one-minute
# response is a namedTuple: https://docs.tweepy.org/en/stable/response.html
# response.data is a dict

# Setup the the tweepy model class to return a json str serialization of the response.data dictionary

class Mytweetlogger(tweepy.StreamingClient):

    def on_connect(self):
        self.start_time = time.time()
        self.timelimit = 60*60 # Set timelimit in seconds
        self.tweetlimit = 100000 # Set tweet limit
        self.saveFile = open('tweets_sentiment.txt', 'a') # Set output file name to append results to
        self.tweet_count = 0
        return super().on_connect()

    def on_response(self, response):
        if ((time.time() - self.start_time) < self.timelimit) and (self.tweet_count < self.tweetlimit):
            data = json.dumps(dict(response.data))
            self.tweet_count += 1
            print(f"#{str(self.tweet_count)}: ", data)
            self.saveFile.write(data)
            self.saveFile.write("\n")
        else:
            self.saveFile.close()
            print(f"Timeout reached. {str(self.tweet_count)} Tweets appended to the record. Disconnecting.")
            self.disconnect()


twitter_stream_sentiment = Mytweetlogger(bearer_token)

# Delete all existing stream rules
existing_rules = twitter_stream_sentiment.get_rules()
print(existing_rules)
if existing_rules.data != None:
    for streamrule in existing_rules.data:
        twitter_stream.delete_rules(streamrule.id)

# Add stream rules
twitter_stream_sentiment.add_rules([tweepy.StreamRule('(":)" OR ":(") lang:en')])
existing_rules = twitter_stream_sentiment.get_rules()
print(existing_rules)

# Start capturing tweets
twitter_stream_sentiment.filter()

Response(data=None, includes={}, errors=[], meta={'sent': '2022-10-29T02:42:04.951Z', 'result_count': 0})


Stream encountered HTTP error: 503
HTTP error response text: {"title":"ConnectionException","detail":"Your subscription change is currently being provisioned, please try again in a minute.","connection_issue":"ProvisioningSubscription","type":"https://api.twitter.com/2/problems/streaming-connection"}


Response(data=[StreamRule(value='(":)" OR ":(") lang:en', tag=None, id='1586186578921402369')], includes={}, errors=[], meta={'sent': '2022-10-29T02:42:05.268Z', 'result_count': 1})
#1:  {"edit_history_tweet_ids": [1586186665110147072], "id": 1586186665110147072, "text": "RT @btssomma: Seokjin came from Sk to Argentina, came this long.. Because he wanted to sing this song for armys, to meet them with this bea\u2026"}
#2:  {"edit_history_tweet_ids": [1586186667257253888], "id": 1586186667257253888, "text": "RT @milephakphum: ,,, Have A GN My GreenyRose(s) &amp; All Cuties :) ,,,\n\n.\n.\n#GreenyRose \n#MilePhakphum https://t.co/P4QjNBS9An"}
#3:  {"edit_history_tweet_ids": [1586186667928346625], "id": 1586186667928346625, "text": "RT @clowncakesoup: tiny dog + a dandelion :) \ud83c\udf3c https://t.co/aqOx5EfQPs"}
#4:  {"edit_history_tweet_ids": [1586186668456951811], "id": 1586186668456951811, "text": "RT @jikookalbum: crying :( https://t.co/WQKuds5yCV"}
#5:  {"edit_history_tweet_ids": [

KeyboardInterrupt: 

**Task:** Edit function `main` so it can store tweets anywhere (location specified as parameter). The FILTER and LANGUAGES should be parameters as well. Test it with different values and languages.

**Task:** Create File `stream_tweets.py` that can be executed from the Terminal by exporting the code from this notebook.

**Task:** Start storing tweets which contain either happy smiley (`:)`) or sad smiley (`:(`) in their text. We will use this dataset during the NLP section. It's good to let the script running for at least 2-3 hours to collect enough data for future modeling.

> #### Note
> the main function runs as an ongoing process and won;t stop until you stop it!